# Two-particle self consistency (TPSC)

Now, we use the Lindhard function to solve the TPSC approximation explained in the lecture notes
of André-Marie Tremblay and in the following [review](https://arxiv.org/abs/1107.1534). 

In TPSC, the vertices for charge and spin fluctuations are different but are still local, i.e. momentum and frequency independent like the bare $U$ in the Hubbard model. This allows conservations laws and the Pauli principle to be satisfied. 

More specifically, the spin and charge susceptibilities are 

$$ \chi_{sp}(\mathbf{q}, iq_n) \equiv \frac{\chi_0(\mathbf{q}, iq_n)}{1 - \frac{U_{sp}}{2} \chi_0(\mathbf{q}, iq_n)} $$

$$ \chi_{ch}(\mathbf{q}, iq_n) \equiv \frac{\chi_0(\mathbf{q}, iq_n)}{1 + \frac{U_{ch}}{2} \chi_0(\mathbf{q}, iq_n)} $$

The sum over all momenta and frequencies of the spin susceptibility gives the equal-time equal-position correlation function. This gives the sum-rule  

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \chi_{sp}(\mathbf{q}, iq_n)=\left< (n_\uparrow - n_\downarrow)^2\right>=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

because the Pauli principle requires that $\left< n_\uparrow^2\right>=\left< n_\uparrow\right>$ since the occupation number on a site is either 0 or 1.

Substituting the TPSC value of the spin susceptibility, double occupancy and $U_{sp}$ can be determined from

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},iq_n)}=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

with the ansatz

\begin{equation}
U_{sp}\left<n_\uparrow\right> \left<n_\downarrow\right>=U\left<n_\uparrow n_\downarrow\right>.
\end{equation}

Given the double occupancy, the charge vertex can then be obtained from

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U_{cn}}{2}\chi_0(\mathbf{q},iq_n)}=n+2\left< n_\uparrow n_\downarrow\right>-n^2.
\end{equation}

Note that the sums over $\mathbf{q}$ and $i\omega_n$ can be interpreted as a trace.

In [ ]:
# Imports 
%matplotlib inline
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, inverse
from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot
import numpy as np
from math import cos, pi

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figuresfrom pytriqs.archive import HDFArchivefrom pytriqs.gf import Gf, inverse

from scipy.optimize import fsolve

In [ ]:
#reload chi0
with HDFArchive("tpsc.h5", 'r') as R:
    chi0_kw = R['chi0_kw']

In [ ]:
def chi_rpa(chi0_kw, U):
    """Compute chi_rpa from chi"""
    chi = chi0_kw.copy()
    return chi * inverse(1 - float(U) * chi)

def trace_chi_kw(chi):
    """Given chi_kw, it computes sum_k sum_\nu chi(k,\nu)""" 
    kmesh, wmesh = chi.mesh.components
    trace = chi.data.sum() / len(kmesh) / wmesh.beta
    # v1 : with density, v2 : without, simple sum
    #trace1 = sum(chi (k.value, all).density() for k in kmesh)/len(kmesh) 
    #print "diff trace ", abs(trace + trace1), trace.real, trace1.real, abs(trace + trace1)/abs(trace)
    assert(np.abs(trace.imag) < 1e-10), trace.imag
    return trace.real

def Usp_root_problem(Usp, chi0, n, U):
    """Sets the self-consistency for U_sp as the problem of finding roots"""
    tr_chi_sp = trace_chi_kw(chi_rpa(chi0, U=Usp[0]))
    diff = 2*tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

def Uch_root_problem(Uch, chi0, n, U, docc):
    """Sets the self-consistency for U_ch as the problem of finding roots"""
    tr_chi_ch = trace_chi_kw(chi_rpa(chi0, U=-Uch[0]))
    diff = 2*tr_chi_ch - 2 * docc - n + n**2
    return diff

def solve_Usp_and_Uch(chi0, U, n, Usp0=0.1, Uch0=0.1):
    """Solves the root problem for U_sp and U_ch and outputs double occupancy as well"""
    Usp = fsolve(Usp_root_problem, Usp0, args=(chi0, n, U), xtol=1e-2)[0]
    docc = 0.25 * Usp / U * n**2
    Uch = fsolve(Uch_root_problem, Uch0, args=(chi0, n, U, docc), xtol=1e-2)[0]
    return Usp, Uch, docc

## Reproduce paper figure

We want to reproduce the following figure from the
__[paper](https://jp1.journaldephysique.org/articles/jp1/abs/1997/11/jp1v7p1309/jp1v7p1309.html)__

<img src="./img/Fig2.png" alt="Drawing" style="width: 250px;"/>

by calling the function tpsc for a grid of values of the bare U.


In [ ]:
n = 1.0
Usp, Uch = 0.3, 0.3

# Initializes a table to store the results
U_vec = np.concatenate((np.arange(0.3, 1., 0.2), np.arange(1., 6., 1.)))    
Usp_vec, Uch_vec, docc_vec = [np.zeros_like(U_vec) for x in xrange(3)]

# Printing the header of the table
print ''.join('| %-11s' % s for s in ['n', 'U', 'Usp', 'Uch', 'docc']), '|'
print '-'*67

# Loop over the different values of bare U
for idx, U in enumerate(U_vec):
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_kw, U, n, Usp0=Usp, Uch0=Uch)
    Usp_vec[idx], Uch_vec[idx], docc_vec[idx] = Usp, Uch, docc
    print ''.join('| %4.4E ' % x for x in [n, U, Usp, Uch, docc]), '|'

## Make plot and compare with paper results

In [ ]:
from matplotlib.image import imread

# Some manual adjustements here to overlay the original figure. May change from one
# machine to another
im = imread("img/Fig2.png")
plt.imshow(im, extent=(-0.45, 5.35, -3, 20.5), aspect='auto')
plt.plot([0,5,5,0,0],[0,0,20,20,0],'-r')

plt.plot(U_vec, Usp_vec, 'o-', label=r'$U_{sp}$', alpha=1, lw=2)
plt.plot(U_vec, Uch_vec, 'o-', label=r'$U_{ch}$', alpha=1, lw=2)

plt.axis('off');

### Note: weakness of RPA

Given the above sum rules, note that in TPSC the following sum-rule, a consequence of the Pauli principle, is satisfied:

\begin{equation}
2\frac{T}{N}\sum_{\mathbf{q},iq_n} \left (\frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},iq_n)}+\frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U_{ch}}{2}\chi_0(\mathbf{q},iq_n)}\right)=2n-n^2.
\end{equation}

Note that the right-hand side is independent of interactions.

    Compute the same quantity, but in the RPA approximation, 
    and make a plot of how much RPA violates the Pauli principle.

In RPA the left hand of the above equation takes the form

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \left (\frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U}{2}\chi_0(\mathbf{q},iq_n)}+\frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U}{2}\chi_0(\mathbf{q},iq_n)}\right)
\end{equation}

Note that the bare $U$ enters in both denominators. Compute this and compare with the relation $2n - n^2$ required by the Pauli principle.

In [ ]:
import numpy as np
from pytriqs.gf import Idx

n = 1.0
U_vec = np.arange(0., 2.5, 0.25)
sum_chi_vec = np.zeros_like(U_vec)

kmesh, wmesh = chi0_kw.mesh.components

# Printing the header of the table
print ''.join('| %-11s' % s for s in ['U', 'sum_chi', '2n-n*n']), '|'
print '-'*41

# Loop over the different values of U
for idx, U in enumerate(U_vec):
    sum_chi = chi_rpa(chi0_kw, +0.5*U) + chi_rpa(chi0_kw, -0.5*U)
    sum_chi = sum_chi.data.sum() / len(kmesh) / wmesh.beta
    sum_chi_vec[idx] = sum_chi.real
    print ''.join('| %4.4E ' % x for x in [U, sum_chi.real, 2*n-n**2]), '|'

In [ ]:
plt.plot(U_vec, sum_chi_vec, '.-', label=r'Tr[$\chi_{ch} + \chi_{sp}$]')
plt.plot(U_vec, 0*U_vec + 2*n - n**2, 'k-', lw=0.5, label=r'$2n-n^2$')
plt.ylim([0.9, 1.2])
plt.xlabel(r'$U$')
plt.legend()